# DATA ASSURANCE

In [15]:
# Load post-imputation dataset and initialize quality assessment tools
import os
import pandas as pd
from pathlib import Path
from scipy.stats import zscore

# Fix path to go up one directory level from notebooks/ to project root
data_folder = Path.cwd().parent / 'data'
data_file = data_folder / 'my_clean_data_with_imputation.csv'
if data_file.exists():
    df = pd.read_csv(data_file)
    print(f"Successfully loaded dataset from: {data_file}")
else:
    raise FileNotFoundError(f"File not found: {data_file}")

# Process data by day using day_id
if 'day_id' in df.columns:
    print(f"Processing data by day using day_id column")
    unique_days = sorted(df['day_id'].unique())
    print(f"Number of unique days: {len(unique_days)}")
else:
    print("Warning: No day_id column found in dataset")
    unique_days = []

Successfully loaded dataset from: c:\Users\13min\Final-Group-ML-Project-Theme-5\data\my_clean_data_with_imputation.csv
Processing data by day using day_id column
Number of unique days: 15


## Dataset Dimensions Check

In [16]:
# Verify dataset size and structure for baseline understanding
print("Initial dataset shape:", df.shape)

Initial dataset shape: (4987646, 16)


## Duplicate Row Detection

In [17]:
# Identify exact duplicate rows that may inflate model performance
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

# Process by day if day_id column exists
if 'day_id' in df.columns and len(unique_days) > 0:
    print("\nDuplicates by day:")
    for day in unique_days:
        day_data = df[df['day_id'] == day]
        day_duplicates = day_data.duplicated().sum()
        print(f"Day {day}: {day_duplicates} duplicates")

Duplicate rows: 2548494

Duplicates by day:
Day 2022.0-7-11: 171737 duplicates
Day 2022.0-7-12: 177688 duplicates
Day 2022.0-7-13: 137569 duplicates
Day 2022.0-7-12: 177688 duplicates
Day 2022.0-7-13: 137569 duplicates
Day 2022.0-7-14: 156841 duplicates
Day 2022.0-7-15: 157606 duplicates
Day 2022.0-7-14: 156841 duplicates
Day 2022.0-7-15: 157606 duplicates
Day 2022.0-7-18: 147975 duplicates
Day 2022.0-7-19: 120109 duplicates
Day 2022.0-7-18: 147975 duplicates
Day 2022.0-7-19: 120109 duplicates
Day 2022.0-7-20: 146310 duplicates
Day 2022.0-7-21: 164225 duplicates
Day 2022.0-7-20: 146310 duplicates
Day 2022.0-7-21: 164225 duplicates
Day 2022.0-7-22: 165310 duplicates
Day 2022.0-7-22: 165310 duplicates
Day 2022.0-7-4: 232235 duplicates
Day 2022.0-7-5: 216023 duplicates
Day 2022.0-7-4: 232235 duplicates
Day 2022.0-7-5: 216023 duplicates
Day 2022.0-7-6: 222498 duplicates
Day 2022.0-7-7: 165639 duplicates
Day 2022.0-7-6: 222498 duplicates
Day 2022.0-7-7: 165639 duplicates
Day 2022.0-7-8: 166

## GPS Coordinate Validation

In [18]:
# Check for invalid latitude/longitude values outside valid geographic ranges
invalid_gps = df[(df["latitude"] < -90) | (df["latitude"] > 90) | (df["longitude"] < -180) | (df["longitude"] > 180)]
print(f"Invalid GPS rows: {len(invalid_gps)}")

# Process by day if day_id column exists
if 'day_id' in df.columns and len(unique_days) > 0:
    print("\nInvalid GPS by day:")
    for day in unique_days:
        day_data = df[df['day_id'] == day]
        day_invalid = day_data[(day_data["latitude"] < -90) | (day_data["latitude"] > 90) | (day_data["longitude"] < -180) | (day_data["longitude"] > 180)]
        print(f"Day {day}: {len(day_invalid)} invalid GPS rows")

Invalid GPS rows: 0

Invalid GPS by day:
Day 2022.0-7-11: 0 invalid GPS rows
Day 2022.0-7-12: 0 invalid GPS rows
Day 2022.0-7-11: 0 invalid GPS rows
Day 2022.0-7-12: 0 invalid GPS rows
Day 2022.0-7-13: 0 invalid GPS rows
Day 2022.0-7-13: 0 invalid GPS rows
Day 2022.0-7-14: 0 invalid GPS rows
Day 2022.0-7-15: 0 invalid GPS rows
Day 2022.0-7-14: 0 invalid GPS rows
Day 2022.0-7-15: 0 invalid GPS rows
Day 2022.0-7-18: 0 invalid GPS rows
Day 2022.0-7-18: 0 invalid GPS rows
Day 2022.0-7-19: 0 invalid GPS rows
Day 2022.0-7-20: 0 invalid GPS rows
Day 2022.0-7-19: 0 invalid GPS rows
Day 2022.0-7-20: 0 invalid GPS rows
Day 2022.0-7-21: 0 invalid GPS rows
Day 2022.0-7-21: 0 invalid GPS rows
Day 2022.0-7-22: 0 invalid GPS rows
Day 2022.0-7-4: 0 invalid GPS rows
Day 2022.0-7-22: 0 invalid GPS rows
Day 2022.0-7-4: 0 invalid GPS rows
Day 2022.0-7-5: 0 invalid GPS rows
Day 2022.0-7-5: 0 invalid GPS rows
Day 2022.0-7-6: 0 invalid GPS rows
Day 2022.0-7-7: 0 invalid GPS rows
Day 2022.0-7-6: 0 invalid GPS

## Negative Value Validation

In [19]:
# Detect logically impossible negative values in latency and throughput metrics
neg_latency = (df[["svr1", "svr2", "svr3", "svr4"]] < 0).sum().sum()
neg_throughput = (df[["upload_bitrate_mbits/sec", "download_bitrate_rx_mbits/sec"]] < 0).sum().sum()
print(f"Negative latency values: {neg_latency}")
print(f"Negative throughput values: {neg_throughput}")

# Process by day if day_id column exists
if 'day_id' in df.columns and len(unique_days) > 0:
    print("\nNegative values by day:")
    for day in unique_days:
        day_data = df[df['day_id'] == day]
        day_neg_latency = (day_data[["svr1", "svr2", "svr3", "svr4"]] < 0).sum().sum()
        day_neg_throughput = (day_data[["upload_bitrate_mbits/sec", "download_bitrate_rx_mbits/sec"]] < 0).sum().sum()
        print(f"Day {day}: {day_neg_latency} negative latency, {day_neg_throughput} negative throughput")

Negative latency values: 0
Negative throughput values: 0

Negative values by day:
Day 2022.0-7-11: 0 negative latency, 0 negative throughput
Day 2022.0-7-11: 0 negative latency, 0 negative throughput
Day 2022.0-7-12: 0 negative latency, 0 negative throughput
Day 2022.0-7-13: 0 negative latency, 0 negative throughput
Day 2022.0-7-12: 0 negative latency, 0 negative throughput
Day 2022.0-7-13: 0 negative latency, 0 negative throughput
Day 2022.0-7-14: 0 negative latency, 0 negative throughput
Day 2022.0-7-15: 0 negative latency, 0 negative throughput
Day 2022.0-7-18: 0 negative latency, 0 negative throughput
Day 2022.0-7-14: 0 negative latency, 0 negative throughput
Day 2022.0-7-15: 0 negative latency, 0 negative throughput
Day 2022.0-7-18: 0 negative latency, 0 negative throughput
Day 2022.0-7-19: 0 negative latency, 0 negative throughput
Day 2022.0-7-20: 0 negative latency, 0 negative throughput
Day 2022.0-7-21: 0 negative latency, 0 negative throughput
Day 2022.0-7-19: 0 negative laten

## Define Key Metric Column Groups

In [20]:
# Organize column names for consistent analysis across latency and throughput features
latency_cols = ["svr1", "svr2", "svr3", "svr4"]
throughput_cols = ["upload_bitrate_mbits/sec", "download_bitrate_rx_mbits/sec"]

## Post-Imputation Completeness Verification

In [21]:
# Ensure imputation process successfully eliminated all missing values
missing_any = df.isna().sum().sum()
print(f"Missing values remaining: {missing_any} (should be 0 after imputation)")

Missing values remaining: 0 (should be 0 after imputation)


## Data Type Schema Validation

In [22]:
# Verify critical columns maintain expected numeric data types for ML compatibility
expected_numeric = ["svr1", "svr2", "svr3", "svr4", "upload_bitrate_mbits/sec", "download_bitrate_rx_mbits/sec", "latitude", "longitude"]
type_issues = []
for col in expected_numeric:
    if col in df.columns and not pd.api.types.is_numeric_dtype(df[col]):
        type_issues.append(f"{col}: {df[col].dtype}")
print(f"Non-numeric columns that should be numeric: {type_issues if type_issues else 'None'}")

Non-numeric columns that should be numeric: None


## Feature Variance Assessment

In [23]:
# Identify zero or constant variance columns that provide no predictive value
import numpy as np

numeric_cols = df.select_dtypes(include=[np.number]).columns
zero_var = [col for col in numeric_cols if df[col].nunique() <= 1]
print(f"Zero/constant variance columns: {zero_var if zero_var else 'None'}")

Zero/constant variance columns: None


## Row-Level Anomaly Detection

In [24]:
# Count complete records with extreme values for comprehensive outlier assessment
zscores = df[latency_cols + throughput_cols].apply(zscore)
extreme_rows = (zscores.abs() > 3).any(axis=1).sum()
print(f"Rows with extreme values (z>3): {extreme_rows}")

# Process by day if day_id column exists
if 'day_id' in df.columns and len(unique_days) > 0:
    print("\nExtreme values by day:")
    for day in unique_days:
        day_data = df[df['day_id'] == day]
        day_zscores = day_data[latency_cols + throughput_cols].apply(zscore)
        day_extreme = (day_zscores.abs() > 3).any(axis=1).sum()
        print(f"Day {day}: {day_extreme} rows with extreme values")

print(f"\nDataset ready for EDA: {len(df)} rows, {len(df.columns)} columns")

Rows with extreme values (z>3): 120850

Extreme values by day:
Day 2022.0-7-11: 7364 rows with extreme values
Day 2022.0-7-12: 8094 rows with extreme values
Day 2022.0-7-13: 30326 rows with extreme values
Day 2022.0-7-12: 8094 rows with extreme values
Day 2022.0-7-13: 30326 rows with extreme values
Day 2022.0-7-14: 4750 rows with extreme values
Day 2022.0-7-15: 7324 rows with extreme values
Day 2022.0-7-14: 4750 rows with extreme values
Day 2022.0-7-15: 7324 rows with extreme values
Day 2022.0-7-18: 5802 rows with extreme values
Day 2022.0-7-19: 2468 rows with extreme values
Day 2022.0-7-18: 5802 rows with extreme values
Day 2022.0-7-19: 2468 rows with extreme values
Day 2022.0-7-20: 3466 rows with extreme values
Day 2022.0-7-21: 8328 rows with extreme values
Day 2022.0-7-20: 3466 rows with extreme values
Day 2022.0-7-21: 8328 rows with extreme values
Day 2022.0-7-22: 6250 rows with extreme values
Day 2022.0-7-4: 9588 rows with extreme values
Day 2022.0-7-22: 6250 rows with extreme val

## Remove Duplicate Rows and Save Clean Dataset

In [25]:
# Based on investigation - these are data collection errors, not valid measurements
print("Before duplicate removal:", df.shape)
df_clean = df.drop_duplicates()
print("After duplicate removal:", df_clean.shape)
print(f"Removed {len(df) - len(df_clean)} duplicate rows")

# Process by day and combine knowledge
if 'day_id' in df_clean.columns and len(unique_days) > 0:
    print(f"\nProcessing and combining knowledge by day:")
    daily_stats = []
    for day in unique_days:
        day_data = df_clean[df_clean['day_id'] == day]
        stats = {
            'day': day,
            'records': len(day_data),
            'avg_latency': day_data[latency_cols].mean().mean(),
            'avg_throughput': day_data[throughput_cols].mean().mean()
        }
        daily_stats.append(stats)
        print(f"Day {day}: {len(day_data)} records, avg latency: {stats['avg_latency']:.2f}ms, avg throughput: {stats['avg_throughput']:.2f}Mbps")
    
    # Combine daily knowledge into summary
    daily_df = pd.DataFrame(daily_stats)
    print("\nCombined daily knowledge:")
    print(daily_df)

# Save cleaned dataset to new file for EDA team - fix path to go to project root
output_file = data_folder / "my_clean_data_after_assurance.csv"
df_clean.to_csv(output_file, index=False)
print(f"\n Clean dataset saved to: {output_file}")
print(f" Ready for EDA: {len(df_clean)} rows, {len(df_clean.columns)} columns")
print("\nNext steps:")
print("1. Use the new file for EDA: pd.read_csv('../data/my_clean_data_after_assurance.csv')")
print("2. Original file preserved at: ../data/my_clean_data_with_imputation.csv")

Before duplicate removal: (4987646, 16)
After duplicate removal: (2439152, 16)
Removed 2548494 duplicate rows

Processing and combining knowledge by day:
Day 2022.0-7-11: 170287 records, avg latency: 237.83ms, avg throughput: 12.00Mbps
Day 2022.0-7-12: 157076 records, avg latency: 228.23ms, avg throughput: 12.71Mbps
Day 2022.0-7-13: 136885 records, avg latency: 150.62ms, avg throughput: 17.17Mbps
After duplicate removal: (2439152, 16)
Removed 2548494 duplicate rows

Processing and combining knowledge by day:
Day 2022.0-7-11: 170287 records, avg latency: 237.83ms, avg throughput: 12.00Mbps
Day 2022.0-7-12: 157076 records, avg latency: 228.23ms, avg throughput: 12.71Mbps
Day 2022.0-7-13: 136885 records, avg latency: 150.62ms, avg throughput: 17.17Mbps
Day 2022.0-7-14: 155939 records, avg latency: 273.59ms, avg throughput: 14.02Mbps
Day 2022.0-7-15: 156862 records, avg latency: 261.93ms, avg throughput: 12.19Mbps
Day 2022.0-7-18: 144979 records, avg latency: 291.89ms, avg throughput: 13.1